<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/27_Avg_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SQL**

### Import librares

In [2]:
import pandas as pd

### Set-up for SQL

In [3]:
%load_ext sql
%sql sqlite://

##### Load dataset

In [4]:
monarchs = pd.read_csv('monarchs.csv')
presidents = pd.read_csv('presidents.csv')
prime_minister_terms = pd.read_csv('prime_minister_terms.csv')
prime_ministers = pd.read_csv('prime_ministers.csv')
states = pd.read_csv('states.csv')

##### Upload dataset to SQL

In [5]:
%sql DROP TABLE IF EXISTS monarchs;
%sql --persist monarchs

%sql DROP TABLE IF EXISTS presidents;
%sql --persist presidents

%sql DROP TABLE IF EXISTS prime_minister_terms;
%sql --persist prime_minister_terms

%sql DROP TABLE IF EXISTS prime_ministers;
%sql --persist prime_ministers

%sql DROP TABLE IF EXISTS states;
%sql --persist states

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted states'

### Class work

#### View all table

In [6]:
%%sql SELECT * FROM monarchs LIMIT 1;

 * sqlite://
Done.


index,country,continent,monarch
0,Brunei,Asia,Hassanal Bolkiah


In [7]:
%%sql SELECT * FROM presidents LIMIT 1;

 * sqlite://
Done.


index,country,continent,president
0,Egypt,Africa,Abdel Fattah el-Sisi


In [8]:
%%sql SELECT * FROM prime_minister_terms LIMIT 1;

 * sqlite://
Done.


index,prime_minister,pm_start
0,Mostafa Madbouly,2018


In [9]:
%%sql SELECT * FROM prime_ministers LIMIT 1;

 * sqlite://
Done.


index,country,continent,prime_minister
0,Egypt,Africa,Mostafa Madbouly


In [10]:
%%sql SELECT * FROM states LIMIT 1;

 * sqlite://
Done.


index,country,continent,indep_year
0,Australia,Oceania,1901


#### Subquiry & CTE (WITH)

##### Part 1 (*Subquiry*)

In [11]:
%%sql SELECT
    country,
    continent,
    (SELECT p1.prime_minister
     FROM prime_minister_terms AS p1
     INNER JOIN prime_ministers AS p2
     USING(prime_minister)
     WHERE p2.country = s.country
     ORDER BY pm_start
     LIMIT 1) AS prime_minister
FROM states AS s;

 * sqlite://
Done.


country,continent,prime_minister
Australia,Oceania,Scott Morrison
Brunei,Asia,Hassanal Bolkiah
Chile,South America,None
Egypt,Africa,Mostafa Madbouly
India,Asia,Narendra Modi
Norway,Europe,Jonas Gahr Støre
Oman,Asia,Haitham bin Tarik
Pakistan,Asia,Shehbaz Sharif
Portugal,Europe,António Costa
Uruguay,South America,None


In [12]:
%%sql SELECT country,
             continent,
             (SELECT MIN(pm_start)
             FROM prime_minister_terms AS p2
             WHERE p2.prime_minister = p1.prime_minister) AS prime_minister
FROM prime_ministers AS p1;

 * sqlite://
Done.


country,continent,prime_minister
Egypt,Africa,2018
Portugal,Europe,2015
Pakistan,Asia,2022
United Kingdom,Europe,2019
India,Asia,2014
Australia,Oceania,2018
Norway,Europe,2021
Brunei,Asia,1984
Oman,Asia,2020
New Zealand,Oceania,2017


In [13]:
%%sql SELECT country
FROM presidents AS p1
WHERE p1.president IN (SELECT prime_minister
                      FROM prime_ministers AS p2
                      WHERE p1.president=p2.prime_minister
                      AND p1.country=p2.country);

 * sqlite://
Done.


country


In [14]:
%%sql SELECT country
FROM prime_ministers AS p
WHERE (SELECT COUNT(*)
       FROM monarchs AS m
       WHERE m.country = p.country
       AND p.prime_minister = m.monarch);

 * sqlite://
Done.


country
Brunei
Oman


##### Part 2 (*CTE*)

In [15]:
%%sql
WITH a AS (SELECT 1 AS number)
SELECT number FROM a;

 * sqlite://
Done.


number
1


In [16]:
%%sql
WITH a AS (
    SELECT
        country,
        continent,
        (SELECT p1.prime_minister
         FROM prime_minister_terms AS p1
         INNER JOIN prime_ministers AS p2
         USING(prime_minister)
         WHERE p2.country = s.country
         ORDER BY pm_start
         LIMIT 1) AS prime_minister
    FROM states AS s
)
SELECT * FROM a;

 * sqlite://
Done.


country,continent,prime_minister
Australia,Oceania,Scott Morrison
Brunei,Asia,Hassanal Bolkiah
Chile,South America,None
Egypt,Africa,Mostafa Madbouly
India,Asia,Narendra Modi
Norway,Europe,Jonas Gahr Støre
Oman,Asia,Haitham bin Tarik
Pakistan,Asia,Shehbaz Sharif
Portugal,Europe,António Costa
Uruguay,South America,None


In [17]:
%%sql
WITH a AS (
    SELECT country,
             continent,
             (SELECT MIN(pm_start)
             FROM prime_minister_terms AS p2
             WHERE p2.prime_minister = p1.prime_minister) AS prime_minister
    FROM prime_ministers AS p1
)
SELECT * FROM a;

 * sqlite://
Done.


country,continent,prime_minister
Egypt,Africa,2018
Portugal,Europe,2015
Pakistan,Asia,2022
United Kingdom,Europe,2019
India,Asia,2014
Australia,Oceania,2018
Norway,Europe,2021
Brunei,Asia,1984
Oman,Asia,2020
New Zealand,Oceania,2017


In [18]:
%%sql
WITH a AS (
    SELECT country
    FROM presidents AS p1
    WHERE p1.president IN (SELECT prime_minister
                      FROM prime_ministers AS p2
                      WHERE p1.president=p2.prime_minister
                      AND p1.country=p2.country)
)
SELECT * FROM a;

 * sqlite://
Done.


country


In [19]:
%%sql
WITH a AS (
    SELECT country
    FROM prime_ministers AS p
    WHERE (SELECT COUNT(*)
           FROM monarchs AS m
           WHERE m.country = p.country
           AND p.prime_minister = m.monarch)
)
SELECT * FROM a;

 * sqlite://
Done.


country
Brunei
Oman


##### Result

Simple Subquery in SELECT
Write a query to list all countries along with their continent and the earliest pm_start year for each country.
Use a simple subquery within the SELECT clause to find the earliest pm_start for each country.

In [20]:
%%sql SELECT country,
             continent,
             (SELECT MIN(pm_start)
             FROM prime_minister_terms AS p2
             WHERE p2.prime_minister = p1.prime_minister) AS prime_minister
FROM prime_ministers AS p1;

 * sqlite://
Done.


country,continent,prime_minister
Egypt,Africa,2018
Portugal,Europe,2015
Pakistan,Asia,2022
United Kingdom,Europe,2019
India,Asia,2014
Australia,Oceania,2018
Norway,Europe,2021
Brunei,Asia,1984
Oman,Asia,2020
New Zealand,Oceania,2017
